# Comparison of CCT- & CMC-learners and other approaches (simulations)

**On Nie and Wager (2021) synthetic data**

In [1]:

import sys
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import warnings
import pandas as pd
from sklearn.model_selection import train_test_split
from crepes_weighted import WrapRegressor
from tqdm.notebook import tqdm


sys.path.append('../..')
from src.cmc_metalearners.cmc_metalearners import CMC_S_Learner, CMC_T_Learner, CMC_X_Learner, CCT_Learner
from src.wcp.wcp import NaiveWCP, NestedWCP
from src.conformal_metalearners.CM_learner import CM_learner
from src.datasets.nie_wager_synthetic import (simulate_nuisance_and_easy_treatment,
                                             simulate_randomized_trial,
                                             simulate_easy_propensity_difficult_baseline,
                                             simulate_unrelated_treatment_control)

In [2]:
get_save_data = True
heteroscedastic_epsilon = False
NSim = 100
alphas = [0.1]
learner = RandomForestRegressor
learner_name = "RF"
MC_samples = 100
normalized_conformal = True
if normalized_conformal:
    normalized_conformal_name = "normalized"
else:
    normalized_conformal_name = "nonnormalized"
max_min_y = True

In [3]:
for i, setup in enumerate([simulate_nuisance_and_easy_treatment, simulate_randomized_trial, simulate_easy_propensity_difficult_baseline, simulate_unrelated_treatment_control]):
    if i == 0:
        setup_name = "A"
    elif i == 1:
        setup_name = "B"
    elif i == 2:
        setup_name = "C"
    else:
        setup_name = "D"
    for n in tqdm(range(NSim)):
        if heteroscedastic_epsilon:
            path_train = f"../../data/simulations/nie_wager/setup{setup_name}/simulations_{setup_name}_{n}_train_heteroscedastic.csv"
            path_test = f"../../data/simulations/nie_wager/setup{setup_name}/simulations_{setup_name}_{n}_test_heteroscedastic.csv"
        else:
            path_train = f"../../data/simulations/nie_wager/setup{setup_name}/simulations_{setup_name}_{n}_train.csv"
            path_test = f"../../data/simulations/nie_wager/setup{setup_name}/simulations_{setup_name}_{n}_test.csv"
        if get_save_data:
            ds_train = pd.read_csv(path_train)
            ds_test = pd.read_csv(path_test)
            y_train = ds_train["y"].values
            y_test = ds_test["y"].values
            X_train = ds_train[[f"X{i}" for i in range(5)]].values
            X_test = ds_test[[f"X{i}" for i in range(5)]].values
            W_train = ds_train["W"].values
            W_test = ds_test["W"].values
            tau_train = ds_train["tau"].values
            tau_test = ds_test["tau"].values
            b_train = ds_train["b"].values
            b_test = ds_test["b"].values
            ps_train = ds_train["ps"].values
            ps_test = ds_test["ps"].values
            y0_train = ds_train["y0"].values
            y0_test = ds_test["y0"].values
            y1_train = ds_train["y1"].values
            y1_test = ds_test["y1"].values
            ite_train = ds_train["ite"].values
            ite_test = ds_test["ite"].values
        else:
            y, X, treatment, tau, b, e, y0, y1 = setup(n=5000, c=0.0, heteroscedastic=heteroscedastic_epsilon)
            ite = y1 - y0
            ps = e
            (y_train, y_test, X_train, X_test,
            W_train, W_test,
            tau_train, tau_test, b_train, b_test,
            ps_train, ps_test, y0_train, y0_test,
            y1_train, y1_test, ite_train, ite_test) = train_test_split(y, X, treatment, tau, b, ps, y0, y1, ite,
                                                                        test_size=0.5)
            df_X_train = pd.DataFrame(X_train, columns=[f"X{i}" for i in range(X_train.shape[1])])
            df_X_test = pd.DataFrame(X_test, columns=[f"X{i}" for i in range(X_test.shape[1])])
            ds_train = pd.concat((df_X_train, pd.DataFrame({
                "y": y_train,
                "W": W_train,
                "tau": tau_train,
                "b": b_train,
                "ps": ps_train,
                "y0": y0_train,
                "y1": y1_train,
                "ite": ite_train
            })), axis=1)
            ds_test = pd.concat((df_X_test, pd.DataFrame({
                "y": y_test,
                "W": W_test,
                "tau": tau_test,
                "b": b_test,
                "ps": ps_test,
                "y0": y0_test,
                "y1": y1_test,
                "ite": ite_test
            })), axis=1)
            ds_train.to_csv(path_train, index=False)
            ds_test.to_csv(path_test, index=False)
        # Initialize the learner
        conformal_pseudo_MC_T_Learner = CMC_T_Learner(
            learner(),
            learner(),
            normalized_conformal=normalized_conformal,
            pseudo_MC=True,
            MC_samples=MC_samples,
            max_min_y=max_min_y
        )
        conformal_pseudo_MC_T_Learner.fit(X_train, y_train, W_train, ps_train)

        conformal_MC_T_Learner = CMC_T_Learner(
            learner(),
            learner(),
            normalized_conformal=normalized_conformal,
            pseudo_MC=False,
            MC_samples=MC_samples,
            max_min_y=max_min_y
        )
        conformal_MC_T_Learner.fit(X_train, y_train, W_train, ps_train)

        conformal_pseudo_MC_S_Learner = CMC_S_Learner(
            learner(),
            normalized_conformal=normalized_conformal,
            pseudo_MC=True,
            MC_samples=MC_samples,
            max_min_y=max_min_y
        )
        with warnings.catch_warnings():
            conformal_pseudo_MC_S_Learner.fit(X_train, y_train, W_train, ps_train)

        conformal_MC_S_Learner = CMC_S_Learner(
            learner(),
            normalized_conformal=normalized_conformal,
            pseudo_MC=False,
            MC_samples=MC_samples,
            max_min_y=max_min_y
        )

        with warnings.catch_warnings():
            conformal_MC_S_Learner.fit(X_train, y_train, W_train, ps_train)


        conformal_pseudo_MC_X_Learner = CMC_X_Learner(
            learner(),
            learner(),
            learner(),
            normalized_conformal=normalized_conformal,
            pseudo_MC=True,
            MC_samples=MC_samples,
            max_min_y=max_min_y
        )
        # Fit the learner
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            conformal_pseudo_MC_X_Learner.fit(X_train, y_train, W_train, ps_train)
        conformal_MC_X_Learner = CMC_X_Learner(
            learner(),
            learner(),
            learner(),
            normalized_conformal=normalized_conformal,
            pseudo_MC=False,
            MC_samples=MC_samples,
            max_min_y=max_min_y
        )

        # Fit the learner
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            conformal_MC_X_Learner.fit(X_train, y_train, W_train, ps_train)

        conformal_CT_learner = CCT_Learner(learner(), learner(), normalized_conformal=normalized_conformal)
        conformal_CT_learner.fit(X_train, y_train, W_train, p=ps_train)

        naive_WCP = NaiveWCP(
                        learner(),
                        learner(),
                        normalized_conformal=normalized_conformal)


        naive_WCP.fit(X_train, y_train, W_train, ps_train)

        (X_train_nuisance, X_train_cal,
            y_train_nuisance, y_train_cal,
            y0_train_nuisance, y0_train_cal,
            y1_train_nuisance, y1_train_cal,
            W_train_nuisance, W_train_cal,
            ite_train_nuisance, ite_train_cal) = train_test_split(
                X_train, y_train, y0_train, y1_train, W_train, ite_train, test_size=0.5, random_state=n
        )

        conformal_y0 = WrapRegressor(learner())
        conformal_y0.fit(X_train_nuisance, y0_train_nuisance)
        conformal_y0.calibrate(X_train_cal, y0_train_cal, cps=True)
        conformal_y1 = WrapRegressor(learner())
        conformal_y1.fit(X_train_nuisance, y1_train_nuisance)
        conformal_y1.calibrate(X_train_cal, y1_train_cal, cps=True)
        conformal_ite = WrapRegressor(learner())
        conformal_ite.fit(X_train_nuisance, ite_train_nuisance)
        conformal_ite.calibrate(X_train_cal, ite_train_cal, cps=True)
        conformal_y_cond0 = WrapRegressor(learner())
        conformal_y_cond0.fit(X_train_nuisance[W_train_nuisance == 0], y_train_nuisance[W_train_nuisance == 0])
        conformal_y_cond0.calibrate(X_train_cal[W_train_cal == 0], y_train_cal[W_train_cal == 0], cps=True)
        conformal_y_cond1 = WrapRegressor(learner())
        conformal_y_cond1.fit(X_train_nuisance[W_train_nuisance == 1], y_train_nuisance[W_train_nuisance == 1])
        conformal_y_cond1.calibrate(X_train_cal[W_train_cal == 1], y_train_cal[W_train_cal == 1], cps=True)

        int_results_y0 = []
        int_results_y1 = []
        int_results_pseudo_MC_T_ite = []
        int_results_MC_T_ite = []
        int_results_pseudo_MC_S_ite = []
        int_results_MC_S_ite = []
        int_results_pseudo_MC_X_ite = []
        int_results_MC_X_ite = []
        int_results_CT_ite = []
        int_results_CM_ite = []
        int_results_naive_ite = []
        int_results_exact_ite = []
        int_results_inexact_ite = []
        int_results_y0_oracle = []
        int_results_y1_oracle = []
        int_results_ite_oracle = []
        int_results_y_cond0 = []
        int_results_y_cond1 = []

        coverage_y0 = []
        coverage_y1 = []
        coverage_pseudo_MC_T_ite = []
        coverage_MC_T_ite = []
        coverage_pseudo_MC_S_ite = []
        coverage_MC_S_ite = []
        coverage_pseudo_MC_X_ite = []
        coverage_MC_X_ite = []
        coverage_CT_ite = []
        coverage_CM_ite = []
        coverage_naive_ite = []
        coverage_exact_ite = []
        coverage_inexact_ite = []
        coverage_y0_oracle = []
        coverage_y1_oracle = []
        coverage_ite_oracle = []
        coverage_y_cond0 = []
        coverage_y_cond1 = []

        int_width_y0 = []
        int_width_y1 = []
        int_width_pseudo_MC_T = []
        int_width_MC_T = []
        int_width_pseudo_MC_S = []
        int_width_MC_S = []
        int_width_pseudo_MC_X = []
        int_width_MC_X = []
        int_width_CT = []
        int_width_CM = []
        int_width_naive_ite = []
        int_width_exact_ite = []
        int_width_inexact_ite = []
        int_width_y0_oracle = []
        int_width_y1_oracle = []
        int_width_ite_oracle = []
        int_width_y_cond0 = []
        int_width_y_cond1 = []


        rmse_y0 = []
        rmse_y1 = []
        rmse_pseudo_MC_T_ite, rmse_pseudo_MC_T_ite_conformal_mean = [], []
        rmse_MC_T_ite,  rmse_MC_T_ite_conformal_mean = [], []
        rmse_pseudo_MC_S_ite, rmse_pseudo_MC_S_ite_conformal_mean = [], []
        rmse_MC_S_ite, rmse_MC_S_ite_conformal_mean = [], []
        rmse_pseudo_MC_X_ite, rmse_pseudo_MC_X_ite_conformal_mean = [], []
        rmse_MC_X_ite, rmse_MC_X_ite_conformal_mean = [], []
        rmse_CT_ite = []
        rmse_CM_ite = []
        rmse_naive_ite = []
        rmse_exact_ite = []
        rmse_inexact_ite = []
        rmse_y0_oracle = []
        rmse_y1_oracle = []
        rmse_ite_oracle = []
        rmse_y_cond0 = []
        rmse_y_cond1 = []


        for alpha in alphas:
            int_pseudo_MC_ite_test = conformal_pseudo_MC_T_Learner.predict_int(X_test, confidence=1-alpha)
            int_MC_ite_test = conformal_MC_T_Learner.predict_int(X_test, confidence=1-alpha)
            int_pseudo_MC_S_ite_test = conformal_pseudo_MC_S_Learner.predict_int(X_test, confidence=1-alpha)
            int_MC_S_ite_test = conformal_MC_S_Learner.predict_int(X_test, confidence=1-alpha)
            int_pseudo_MC_X_ite_test = conformal_pseudo_MC_X_Learner.predict_int(X_test, confidence=1-alpha)
            int_MC_X_ite_test = conformal_MC_X_Learner.predict_int(X_test, confidence=1-alpha)
            int_CT_ite_test = conformal_CT_learner.predict_int(X_test, confidence=1-alpha, p=ps_test)
            int_y0_test = conformal_CT_learner.predict_int_y0(X_test, confidence=1-alpha, p=ps_test)
            int_y1_test = conformal_CT_learner.predict_int_y1(X_test, confidence=1-alpha, p=ps_test)
            conformal_Learner = CM_learner(metalearner="DR", alpha=alpha)
            conformal_Learner.fit(X_train, y_train, W_train, ps_train)
            int_CM_ite_test = conformal_Learner.predict_int(X_test)
            int_naive_ite_test = naive_WCP.predict_int(X_test, ps_test, confidence=1-alpha)
            exact_wcp = NestedWCP(
                            learner(),
                            learner(),
                            learner(),
                            normalized_conformal=normalized_conformal,
                            exact=True)
            exact_wcp.fit(X_train, y_train, W_train, ps_train, confidence=1-alpha)
            int_exact_ite_test = exact_wcp.predict_int(X_test, ps_test)
            inexact_wcp = NestedWCP(
                            learner(),
                            learner(),
                            normalized_conformal=normalized_conformal,
                            exact=False)
            inexact_wcp.fit(X_train, y_train, W_train, ps_train, confidence=1-alpha)
            int_inexact_ite_test = inexact_wcp.predict_int(X_test, ps_test)
            int_y0_oracle = conformal_y0.predict_int(X_test, confidence=1-alpha)
            int_y1_oracle = conformal_y1.predict_int(X_test, confidence=1-alpha)
            int_ite_oracle = conformal_ite.predict_int(X_test, confidence=1-alpha)
            int_y_cond0 = conformal_y_cond0.predict_int(X_test[W_test == 0], confidence=1-alpha)
            int_y_cond1 = conformal_y_cond1.predict_int(X_test[W_test == 1], confidence=1-alpha)

            int_results_y0.append(int_y0_test)
            int_results_y1.append(int_y1_test)
            int_results_pseudo_MC_T_ite.append(int_pseudo_MC_ite_test)
            int_results_MC_T_ite.append(int_MC_ite_test)
            int_results_pseudo_MC_S_ite.append(int_pseudo_MC_S_ite_test)
            int_results_MC_S_ite.append(int_MC_S_ite_test)
            int_results_pseudo_MC_X_ite.append(int_pseudo_MC_X_ite_test)
            int_results_MC_X_ite.append(int_MC_X_ite_test)
            int_results_CT_ite.append(int_CT_ite_test)
            int_results_CM_ite.append(int_CM_ite_test)
            int_results_naive_ite.append(int_naive_ite_test)
            int_results_exact_ite.append(int_exact_ite_test)
            int_results_inexact_ite.append(int_inexact_ite_test)
            int_results_y0_oracle.append(int_y0_oracle)
            int_results_y1_oracle.append(int_y1_oracle)
            int_results_ite_oracle.append(int_ite_oracle)
            int_results_y_cond0.append(int_y_cond0)
            int_results_y_cond1.append(int_y_cond1)

            coverage_y0.append(np.mean((int_y0_test[:, 0] < y0_test) &  (int_y0_test[:, 1] > y0_test)))
            coverage_y1.append(np.mean((int_y1_test[:, 0] < y1_test) &  (int_y1_test[:, 1] > y1_test)))
            coverage_pseudo_MC_T_ite.append(np.mean((int_pseudo_MC_ite_test[:, 0] < ite_test) &  (int_pseudo_MC_ite_test[:, 1] > ite_test)))
            coverage_MC_T_ite.append(np.mean((int_MC_ite_test[:, 0] < ite_test) &  (int_MC_ite_test[:, 1] > ite_test)))
            coverage_pseudo_MC_S_ite.append(np.mean((int_pseudo_MC_S_ite_test[:, 0] < ite_test) &  (int_pseudo_MC_S_ite_test[:, 1] > ite_test)))
            coverage_MC_S_ite.append(np.mean((int_MC_S_ite_test[:, 0] < ite_test) &  (int_MC_S_ite_test[:, 1] > ite_test)))
            coverage_pseudo_MC_X_ite.append(np.mean((int_pseudo_MC_X_ite_test[:, 0] < ite_test) &  (int_pseudo_MC_X_ite_test[:, 1] > ite_test)))
            coverage_MC_X_ite.append(np.mean((int_MC_X_ite_test[:, 0] < ite_test) &  (int_MC_X_ite_test[:, 1] > ite_test)))
            coverage_CT_ite.append(np.mean((int_CT_ite_test[:, 0] < ite_test) &  (int_CT_ite_test[:, 1] > ite_test)))
            coverage_CM_ite.append(np.mean((int_CM_ite_test[:, 0] < ite_test) &  (int_CM_ite_test[:, 1] > ite_test)))
            coverage_naive_ite.append(np.mean((int_naive_ite_test[:, 0] < ite_test) &  (int_naive_ite_test[:, 1] > ite_test)))
            coverage_exact_ite.append(np.mean((int_exact_ite_test[:, 0] < ite_test) &  (int_exact_ite_test[:, 1] > ite_test)))
            coverage_inexact_ite.append(np.mean((int_inexact_ite_test[:, 0] < ite_test) &  (int_inexact_ite_test[:, 1] > ite_test)))
            coverage_y0_oracle.append(np.mean((int_y0_oracle[:, 0] < y0_test) &  (int_y0_oracle[:, 1] > y0_test)))
            coverage_y1_oracle.append(np.mean((int_y1_oracle[:, 0] < y1_test) &  (int_y1_oracle[:, 1] > y1_test)))
            coverage_ite_oracle.append(np.mean((int_ite_oracle[:, 0] < ite_test) &  (int_ite_oracle[:, 1] > ite_test)))
            coverage_y_cond0.append(np.mean((int_y_cond0[:, 0] < y_test[W_test == 0]) &  (int_y_cond0[:, 1] > y_test[W_test == 0])))
            coverage_y_cond1.append(np.mean((int_y_cond1[:, 0] < y_test[W_test == 1]) &  (int_y_cond1[:, 1] > y_test[W_test == 1])))

            int_width_y0.append(np.diff(int_y0_test).mean())
            int_width_y1.append(np.diff(int_y1_test).mean())
            int_width_pseudo_MC_T.append(np.diff(int_pseudo_MC_ite_test).mean())
            int_width_MC_T.append(np.diff(int_MC_ite_test).mean())
            int_width_pseudo_MC_S.append(np.diff(int_pseudo_MC_S_ite_test).mean())
            int_width_MC_S.append(np.diff(int_MC_S_ite_test).mean())
            int_width_pseudo_MC_X.append(np.diff(int_pseudo_MC_X_ite_test).mean())
            int_width_MC_X.append(np.diff(int_MC_X_ite_test).mean())
            int_width_CT.append(np.diff(int_CT_ite_test).mean())
            int_width_CM.append(np.diff(int_CM_ite_test).mean())
            int_width_naive_ite.append(np.diff(int_naive_ite_test).mean())
            int_width_exact_ite.append(np.diff(int_exact_ite_test).mean())
            int_width_inexact_ite.append(np.diff(int_inexact_ite_test).mean())
            int_width_y0_oracle.append(np.diff(int_y0_oracle).mean())
            int_width_y1_oracle.append(np.diff(int_y1_oracle).mean())
            int_width_ite_oracle.append(np.diff(int_ite_oracle).mean())
            int_width_y_cond0.append(np.diff(int_y_cond0).mean())
            int_width_y_cond1.append(np.diff(int_y_cond1).mean())

            rmse_y0.append(np.sqrt(np.mean((y0_test - conformal_CT_learner.predict_y0(X_test, ps_test))**2)))
            rmse_y1.append(np.sqrt(np.mean((y1_test - conformal_CT_learner.predict_y1(X_test, ps_test))**2)))
            rmse_pseudo_MC_T_ite.append(np.sqrt(np.mean((ite_test - conformal_pseudo_MC_T_Learner.predict(X_test))**2)))
            rmse_MC_T_ite.append(np.sqrt(np.mean((ite_test - conformal_MC_T_Learner.predict(X_test))**2)))
            rmse_pseudo_MC_S_ite.append(np.sqrt(np.mean((ite_test - conformal_pseudo_MC_S_Learner.predict(X_test))**2)))
            rmse_MC_S_ite.append(np.sqrt(np.mean((ite_test - conformal_MC_S_Learner.predict(X_test))**2)))
            rmse_pseudo_MC_X_ite.append(np.sqrt(np.mean((ite_test - conformal_pseudo_MC_X_Learner.predict(X_test))**2)))
            rmse_MC_X_ite.append(np.sqrt(np.mean((ite_test - conformal_MC_X_Learner.predict(X_test))**2)))
            rmse_CT_ite.append(np.sqrt(np.mean((ite_test - conformal_CT_learner.predict(X_test, ps_test))**2)))
            rmse_CM_ite.append(np.sqrt(np.mean((ite_test - conformal_Learner.predict(X_test))**2)))
            rmse_naive_ite.append(np.sqrt(np.mean((ite_test - naive_WCP.predict(X_test))**2)))
            rmse_exact_ite.append(np.sqrt(np.mean((ite_test - exact_wcp.predict(X_test))**2)))
            rmse_inexact_ite.append(np.sqrt(np.mean((ite_test - inexact_wcp.predict(X_test))**2)))
            rmse_y0_oracle.append(np.sqrt(np.mean((y0_test - conformal_y0.predict(X_test))**2)))
            rmse_y1_oracle.append(np.sqrt(np.mean((y1_test - conformal_y1.predict(X_test))**2)))
            rmse_ite_oracle.append(np.sqrt(np.mean((ite_test - conformal_ite.predict(X_test))**2)))
            rmse_y_cond0.append(np.sqrt(np.mean((y_test[W_test == 0] - conformal_y_cond0.predict(X_test[W_test == 0]))**2)))
            rmse_y_cond1.append(np.sqrt(np.mean((y_test[W_test == 1] - conformal_y_cond1.predict(X_test[W_test == 1]))**2)))
        df_eval = pd.DataFrame({
            "coverage_y0": coverage_y0,
            "coverage_y1": coverage_y1,
            "coverage_pseudo_MC_T_ite": coverage_pseudo_MC_T_ite,
            "coverage_MC_T_ite": coverage_MC_T_ite,
            "coverage_pseudo_MC_S_ite": coverage_pseudo_MC_S_ite,
            "coverage_MC_S_ite": coverage_MC_S_ite,
            "coverage_pseudo_MC_X_ite": coverage_pseudo_MC_X_ite,
            "coverage_MC_X_ite": coverage_MC_X_ite,
            "coverage_CT_ite": coverage_CT_ite,
            "coverage_CM_ite": coverage_CM_ite,
            "coverage_naive_ite": coverage_naive_ite,
            "coverage_exact_ite": coverage_exact_ite,
            "coverage_inexact_ite": coverage_inexact_ite,
            "coverage_y0_oracle": coverage_y0_oracle,
            "coverage_y1_oracle": coverage_y1_oracle,
            "coverage_ite_oracle": coverage_ite_oracle,
            "coverage_y_cond0": coverage_y_cond0,
            "coverage_y_cond1": coverage_y_cond1,
            "int_width_y0": int_width_y0,
            "int_width_y1": int_width_y1,
            "int_width_pseudo_MC_T": int_width_pseudo_MC_T,
            "int_width_MC_T": int_width_MC_T,
            "int_width_pseudo_MC_S": int_width_pseudo_MC_S,
            "int_width_MC_S": int_width_MC_S,
            "int_width_pseudo_MC_X": int_width_pseudo_MC_X,
            "int_width_MC_X": int_width_MC_X,
            "int_width_CT": int_width_CT,
            "int_width_CM": int_width_CM,
            "int_width_naive_ite": int_width_naive_ite,
            "int_width_exact_ite": int_width_exact_ite,
            "int_width_inexact_ite": int_width_inexact_ite,
            "int_width_y0_oracle": int_width_y0_oracle,
            "int_width_y1_oracle": int_width_y1_oracle,
            "int_width_ite_oracle": int_width_ite_oracle,
            "int_width_y_cond0": int_width_y_cond0,
            "int_width_y_cond1": int_width_y_cond1,
            "rmse_y0": rmse_y0,
            "rmse_y1": rmse_y1,
            "rmse_pseudo_MC_T_ite": rmse_pseudo_MC_T_ite,
            "rmse_MC_T_ite": rmse_MC_T_ite,
            "rmse_pseudo_MC_S_ite": rmse_pseudo_MC_S_ite,
            "rmse_MC_S_ite": rmse_MC_S_ite,
            "rmse_pseudo_MC_X_ite": rmse_pseudo_MC_X_ite,
            "rmse_MC_X_ite": rmse_MC_X_ite,
            "rmse_CT_ite": rmse_CT_ite,
            "rmse_CM_ite": rmse_CM_ite,
            "rmse_naive_ite": rmse_naive_ite,
            "rmse_exact_ite": rmse_exact_ite,
            "rmse_inexact_ite": rmse_inexact_ite,
            "rmse_y0_oracle": rmse_y0_oracle,
            "rmse_y1_oracle": rmse_y1_oracle,
            "rmse_ite_oracle": rmse_ite_oracle,
            "rmse_y_cond0": rmse_y_cond0,
            "rmse_y_cond1": rmse_y_cond1,
            "alpha": alphas
        })
        if max_min_y:
            if heteroscedastic_epsilon:
                df_eval.to_csv(f"../../results/outputs/nie_wager/setup{setup_name}/eval/simulations_{setup_name}_{str(n)}_{learner_name}_{normalized_conformal_name}_max_min_y_heteroscedastic_eval.csv", index=False)
            else:
                df_eval.to_csv(f"../../results/outputs/nie_wager/setup{setup_name}/eval/simulations_{setup_name}_{str(n)}_{learner_name}_{normalized_conformal_name}_max_min_y_eval.csv", index=False)
        else:
            if heteroscedastic_epsilon:
                df_eval.to_csv(f"../../results/outputs/nie_wager/setup{setup_name}/eval/simulations_{setup_name}_{str(n)}_{learner_name}_{normalized_conformal_name}_heteroscedastic_eval.csv", index=False)
            else:
                df_eval.to_csv(f"../../results/outputs/nie_wager/setup{setup_name}/eval/simulations_{setup_name}_{str(n)}_{learner_name}_{normalized_conformal_name}_eval.csv", index=False)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

## Evaluating Probability Calibrations

In [4]:
get_save_data = True

for i, setup in enumerate([simulate_nuisance_and_easy_treatment, simulate_randomized_trial, simulate_easy_propensity_difficult_baseline, simulate_unrelated_treatment_control]):
    list_p_values_pseudo_MC_T = []
    list_p_values_MC_T = []
    list_p_values_pseudo_MC_S = []
    list_p_values_MC_S = []
    list_p_values_pseudo_MC_X = []
    list_p_values_MC_X = []
    list_p_values_CT = []
    list_p_values_y0 = []
    list_p_values_y1 = []
    list_p_values_oracle = []

    if i == 0:
        setup_name = "A"
    elif i == 1:
        setup_name = "B"
    elif i == 2:
        setup_name = "C"
    else:
        setup_name = "D"
    for n in tqdm(range(NSim)):
        # Generate data
        if get_save_data:
            if heteroscedastic_epsilon:
                path_train = f"../../data/simulations/nie_wager/setup{setup_name}/simulations_{setup_name}_{n}_train_heteroscedastic.csv"
                path_test = f"../../data/simulations/nie_wager/setup{setup_name}/simulations_{setup_name}_{n}_test_heteroscedastic.csv"
            else:
                path_train = f"../../data/simulations/nie_wager/setup{setup_name}/simulations_{setup_name}_{n}_train.csv"
                path_test = f"../../data/simulations/nie_wager/setup{setup_name}/simulations_{setup_name}_{n}_test.csv"
            ds_train = pd.read_csv(path_train)
            ds_test = pd.read_csv(path_test)
            y_train = ds_train["y"].values
            y_test = ds_test["y"].values
            X_train = ds_train[[f"X{i}" for i in range(5)]].values
            X_test = ds_test[[f"X{i}" for i in range(5)]].values
            W_train = ds_train["W"].values
            W_test = ds_test["W"].values
            tau_train = ds_train["tau"].values
            tau_test = ds_test["tau"].values
            b_train = ds_train["b"].values
            b_test = ds_test["b"].values
            ps_train = ds_train["ps"].values
            ps_test = ds_test["ps"].values
            y0_train = ds_train["y0"].values
            y0_test = ds_test["y0"].values
            y1_train = ds_train["y1"].values
            y1_test = ds_test["y1"].values
            ite_train = ds_train["ite"].values
            ite_test = ds_test["ite"].values
        else:
            y, X, treatment, tau, b, e, y0, y1 = setup(n=5000, c=0.0, heteroscedastic=heteroscedastic_epsilon)
            ite = y1 - y0
            ps = e
            (y_train, y_test, X_train, X_test,
            W_train, W_test,
            tau_train, tau_test, b_train, b_test,
            ps_train, ps_test, y0_train, y0_test,
            y1_train, y1_test, ite_train, ite_test) = train_test_split(y, X, treatment, tau, b, ps, y0, y1, ite,
                                                                        test_size=0.5)
            df_X_train = pd.DataFrame(X_train, columns=[f"X{i}" for i in range(X_train.shape[1])])
            df_X_test = pd.DataFrame(X_test, columns=[f"X{i}" for i in range(X_test.shape[1])])
            ds_train = pd.concat((df_X_train, pd.DataFrame({
                "y": y_train,
                "W": W_train,
                "tau": tau_train,
                "b": b_train,
                "ps": ps_train,
                "y0": y0_train,
                "y1": y1_train,
                "ite": ite_train
            })), axis=1)
            ds_test = pd.concat((df_X_test, pd.DataFrame({
                "y": y_test,
                "W": W_test,
                "tau": tau_test,
                "b": b_test,
                "ps": ps_test,
                "y0": y0_test,
                "y1": y1_test,
                "ite": ite_test
            })), axis=1)

        # # Initialize the learner
        conformal_pseudo_MC_T_Learner = CMC_T_Learner(
            learner(),
            learner(),
            normalized_conformal=normalized_conformal,
            pseudo_MC=True,
            MC_samples=MC_samples,
            max_min_y=max_min_y
        )
        conformal_pseudo_MC_T_Learner.fit(X_train, y_train, W_train, ps_train)

        conformal_MC_T_Learner = CMC_T_Learner(
            learner(),
            learner(),
            normalized_conformal=normalized_conformal,
            pseudo_MC=False,
            MC_samples=MC_samples,
            max_min_y=max_min_y
        )
        conformal_MC_T_Learner.fit(X_train, y_train, W_train, ps_train)

        # conformal_pseudo_MC_S_Learner = CMC_S_Learner(
        #     learner(),
        #     normalized_conformal=normalized_conformal,
        #     pseudo_MC=True,
        #     MC_samples=MC_samples,
        #     max_min_y=max_min_y
        # )
        # with warnings.catch_warnings():
        #     # Suppress warning that is thrown saying that calibration example is too small
        #     # However, this is a bug in the crepes library in this case
        #     conformal_pseudo_MC_S_Learner.fit(X_train, y_train, W_train, ps_train)

        # conformal_MC_S_Learner = CMC_S_Learner(
        #     learner(),
        #     normalized_conformal=normalized_conformal,
        #     pseudo_MC=False,
        #     MC_samples=MC_samples,
        #     max_min_y=max_min_y
        # )

        # with warnings.catch_warnings():
        #     conformal_MC_S_Learner.fit(X_train, y_train, W_train, ps_train)


        # conformal_pseudo_MC_X_Learner = CMC_X_Learner(
        #     learner(),
        #     learner(),
        #     learner(),
        #     normalized_conformal=normalized_conformal,
        #     pseudo_MC=True,
        #     MC_samples=MC_samples,
        #     max_min_y=max_min_y
        # )
        # # Fit the learner
        # with warnings.catch_warnings():
        #     warnings.simplefilter("ignore")
        #     conformal_pseudo_MC_X_Learner.fit(X_train, y_train, W_train, ps_train)
        # conformal_MC_X_Learner = CMC_X_Learner(
        #     learner(),
        #     learner(),
        #     learner(),
        #     normalized_conformal=normalized_conformal,
        #     pseudo_MC=False,
        #     MC_samples=MC_samples,
        #     max_min_y=max_min_y
        # )

        # # Fit the learner
        # with warnings.catch_warnings():
        #     warnings.simplefilter("ignore")
        #     conformal_MC_X_Learner.fit(X_train, y_train, W_train, ps_train)

        conformal_CT_learner = CCT_Learner(learner(), learner(), normalized_conformal=normalized_conformal)
        conformal_CT_learner.fit(X_train, y_train, W_train, p=ps_train)
        (X_train_nuisance, X_train_cal,
            y_train_nuisance, y_train_cal,
            y0_train_nuisance, y0_train_cal,
            y1_train_nuisance, y1_train_cal,
            W_train_nuisance, W_train_cal,
            ite_train_nuisance, ite_train_cal) = train_test_split(
                X_train, y_train, y0_train, y1_train, W_train, ite_train, test_size=0.5, random_state=n
        )
        conformal_ite_oracle = WrapRegressor(learner())
        conformal_ite_oracle.fit(X_train_nuisance, ite_train_nuisance)
        conformal_ite_oracle.calibrate(X_train_cal, ite_train_cal, cps=True)
        # p-values
        list_p_values_pseudo_MC_T.append(conformal_pseudo_MC_T_Learner.predict_p_value(X_test, ite_test))
        list_p_values_MC_T.append(conformal_MC_T_Learner.predict_p_value(X_test, ite_test))
        # list_p_values_pseudo_MC_S.append(conformal_pseudo_MC_S_Learner.predict_p_value(X_test, ite_test))
        # list_p_values_MC_S.append(conformal_MC_S_Learner.predict_p_value(X_test, ite_test))
        # list_p_values_pseudo_MC_X.append(conformal_pseudo_MC_X_Learner.predict_p_value(X_test, ite_test))
        # list_p_values_MC_X.append(conformal_MC_X_Learner.predict_p_value(X_test, ite_test))
        list_p_values_CT.append(conformal_CT_learner.predict_p_value(X_test, ite_test, p=ps_test))
        list_p_values_y0.append(conformal_CT_learner.predict_p_value_y0(X_test, y0_test, p=ps_test))
        list_p_values_y1.append(conformal_CT_learner.predict_p_value_y1(X_test, y1_test, p=ps_test))
        list_p_values_oracle.append(conformal_ite_oracle.predict_cps(X_test, y=ite_test))

    dict_p_values = {
            "pseudo_MC_T": np.concatenate(list_p_values_pseudo_MC_T),
            "MC_T": np.concatenate(list_p_values_MC_T),
            # "pseudo_MC_S": np.concatenate(list_p_values_pseudo_MC_S),
            # "MC_S": np.concatenate(list_p_values_MC_S),
            # "pseudo_MC_X": np.concatenate(list_p_values_pseudo_MC_X),
            # "MC_X": np.concatenate(list_p_values_MC_X),
            "CT": np.concatenate(list_p_values_CT),
            "y0": np.concatenate(list_p_values_y0),
            "y1": np.concatenate(list_p_values_y1),
            "oracle": np.concatenate(list_p_values_oracle),
    }
    df_p_values = pd.DataFrame(dict_p_values)
    if max_min_y:
        if heteroscedastic_epsilon:
            df_p_values.to_csv(f"../../results/outputs/nie_wager/setup{setup_name}/p_values/simulations_{setup_name}_{learner_name}_{normalized_conformal_name}_max_min_y_heteroscedastic_p_values.csv", index=False)
        else:
            df_p_values.to_csv(f"../../results/outputs/nie_wager/setup{setup_name}/p_values/simulations_{setup_name}_{learner_name}_{normalized_conformal_name}_max_min_y_p_values.csv", index=False)
    else:
        if heteroscedastic_epsilon:
            df_p_values.to_csv(f"../../results/outputs/nie_wager/setup{setup_name}/p_values/simulations_{setup_name}_{learner_name}_{normalized_conformal_name}_heteroscedastic_p_values.csv", index=False)
        else:
            df_p_values.to_csv(f"../../results/outputs/nie_wager/setup{setup_name}/p_values/simulations_{setup_name}_{learner_name}_{normalized_conformal_name}_p_values.csv", index=False)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [5]:
get_save_data = True

for i, setup in enumerate([simulate_nuisance_and_easy_treatment, simulate_randomized_trial, simulate_easy_propensity_difficult_baseline, simulate_unrelated_treatment_control]):
    list_p_values_pseudo_MC_T = []
    list_p_values_MC_T = []
    list_p_values_pseudo_MC_S = []
    list_p_values_MC_S = []
    list_p_values_pseudo_MC_X = []
    list_p_values_MC_X = []
    list_p_values_CT = []
    list_p_values_y0 = []
    list_p_values_y1 = []
    list_p_values_oracle = []

    if i == 0:
        setup_name = "A"
    elif i == 1:
        setup_name = "B"
    elif i == 2:
        setup_name = "C"
    else:
        setup_name = "D"
    for n in tqdm(range(NSim)):
        # Generate data
        if get_save_data:
            if heteroscedastic_epsilon:
                path_train = f"../../data/simulations/nie_wager/setup{setup_name}/simulations_{setup_name}_{n}_train_heteroscedastic.csv"
                path_test = f"../../data/simulations/nie_wager/setup{setup_name}/simulations_{setup_name}_{n}_test_heteroscedastic.csv"
            else:
                path_train = f"../../data/simulations/nie_wager/setup{setup_name}/simulations_{setup_name}_{n}_train.csv"
                path_test = f"../../data/simulations/nie_wager/setup{setup_name}/simulations_{setup_name}_{n}_test.csv"
            ds_train = pd.read_csv(path_train)
            ds_test = pd.read_csv(path_test)
            y_train = ds_train["y"].values
            y_test = ds_test["y"].values
            X_train = ds_train[[f"X{i}" for i in range(5)]].values
            X_test = ds_test[[f"X{i}" for i in range(5)]].values
            W_train = ds_train["W"].values
            W_test = ds_test["W"].values
            tau_train = ds_train["tau"].values
            tau_test = ds_test["tau"].values
            b_train = ds_train["b"].values
            b_test = ds_test["b"].values
            ps_train = ds_train["ps"].values
            ps_test = ds_test["ps"].values
            y0_train = ds_train["y0"].values
            y0_test = ds_test["y0"].values
            y1_train = ds_train["y1"].values
            y1_test = ds_test["y1"].values
            ite_train = ds_train["ite"].values
            ite_test = ds_test["ite"].values
        else:
            y, X, treatment, tau, b, e, y0, y1 = setup(n=5000, c=0.0, heteroscedastic=heteroscedastic_epsilon)
            ite = y1 - y0
            ps = e
            (y_train, y_test, X_train, X_test,
            W_train, W_test,
            tau_train, tau_test, b_train, b_test,
            ps_train, ps_test, y0_train, y0_test,
            y1_train, y1_test, ite_train, ite_test) = train_test_split(y, X, treatment, tau, b, ps, y0, y1, ite,
                                                                        test_size=0.5)
            df_X_train = pd.DataFrame(X_train, columns=[f"X{i}" for i in range(X_train.shape[1])])
            df_X_test = pd.DataFrame(X_test, columns=[f"X{i}" for i in range(X_test.shape[1])])
            ds_train = pd.concat((df_X_train, pd.DataFrame({
                "y": y_train,
                "W": W_train,
                "tau": tau_train,
                "b": b_train,
                "ps": ps_train,
                "y0": y0_train,
                "y1": y1_train,
                "ite": ite_train
            })), axis=1)
            ds_test = pd.concat((df_X_test, pd.DataFrame({
                "y": y_test,
                "W": W_test,
                "tau": tau_test,
                "b": b_test,
                "ps": ps_test,
                "y0": y0_test,
                "y1": y1_test,
                "ite": ite_test
            })), axis=1)

        # # Initialize the learner
        conformal_pseudo_MC_T_Learner = CMC_T_Learner(
            learner(),
            learner(),
            normalized_conformal=normalized_conformal,
            pseudo_MC=True,
            MC_samples=MC_samples,
            max_min_y=max_min_y
        )
        conformal_pseudo_MC_T_Learner.fit(X_train, y_train, W_train)

        conformal_MC_T_Learner = CMC_T_Learner(
            learner(),
            learner(),
            normalized_conformal=normalized_conformal,
            pseudo_MC=False,
            MC_samples=MC_samples,
            max_min_y=max_min_y
        )
        conformal_MC_T_Learner.fit(X_train, y_train, W_train)

        # conformal_pseudo_MC_S_Learner = CMC_S_Learner(
        #     learner(),
        #     normalized_conformal=normalized_conformal,
        #     pseudo_MC=True,
        #     MC_samples=MC_samples,
        #     max_min_y=max_min_y
        # )
        # with warnings.catch_warnings():
        #     # Suppress warning that is thrown saying that calibration example is too small
        #     # However, this is a bug in the crepes library in this case
        #     conformal_pseudo_MC_S_Learner.fit(X_train, y_train, W_train, ps_train)

        # conformal_MC_S_Learner = CMC_S_Learner(
        #     learner(),
        #     normalized_conformal=normalized_conformal,
        #     pseudo_MC=False,
        #     MC_samples=MC_samples,
        #     max_min_y=max_min_y
        # )

        # with warnings.catch_warnings():
        #     conformal_MC_S_Learner.fit(X_train, y_train, W_train, ps_train)


        # conformal_pseudo_MC_X_Learner = CMC_X_Learner(
        #     learner(),
        #     learner(),
        #     learner(),
        #     normalized_conformal=normalized_conformal,
        #     pseudo_MC=True,
        #     MC_samples=MC_samples,
        #     max_min_y=max_min_y
        # )
        # # Fit the learner
        # with warnings.catch_warnings():
        #     warnings.simplefilter("ignore")
        #     conformal_pseudo_MC_X_Learner.fit(X_train, y_train, W_train, ps_train)
        # conformal_MC_X_Learner = CMC_X_Learner(
        #     learner(),
        #     learner(),
        #     learner(),
        #     normalized_conformal=normalized_conformal,
        #     pseudo_MC=False,
        #     MC_samples=MC_samples,
        #     max_min_y=max_min_y
        # )

        # # Fit the learner
        # with warnings.catch_warnings():
        #     warnings.simplefilter("ignore")
        #     conformal_MC_X_Learner.fit(X_train, y_train, W_train, ps_train)

        conformal_CT_learner = CCT_Learner(learner(), learner(), normalized_conformal=normalized_conformal)
        conformal_CT_learner.fit(X_train, y_train, W_train)
        (X_train_nuisance, X_train_cal,
            y_train_nuisance, y_train_cal,
            y0_train_nuisance, y0_train_cal,
            y1_train_nuisance, y1_train_cal,
            W_train_nuisance, W_train_cal,
            ite_train_nuisance, ite_train_cal) = train_test_split(
                X_train, y_train, y0_train, y1_train, W_train, ite_train, test_size=0.5, random_state=n
        )
        conformal_ite_oracle = WrapRegressor(learner())
        conformal_ite_oracle.fit(X_train_nuisance, ite_train_nuisance)
        conformal_ite_oracle.calibrate(X_train_cal, ite_train_cal, cps=True)
        # p-values
        list_p_values_pseudo_MC_T.append(conformal_pseudo_MC_T_Learner.predict_p_value(X_test, ite_test))
        list_p_values_MC_T.append(conformal_MC_T_Learner.predict_p_value(X_test, ite_test))
        # list_p_values_pseudo_MC_S.append(conformal_pseudo_MC_S_Learner.predict_p_value(X_test, ite_test))
        # list_p_values_MC_S.append(conformal_MC_S_Learner.predict_p_value(X_test, ite_test))
        # list_p_values_pseudo_MC_X.append(conformal_pseudo_MC_X_Learner.predict_p_value(X_test, ite_test))
        # list_p_values_MC_X.append(conformal_MC_X_Learner.predict_p_value(X_test, ite_test))
        list_p_values_CT.append(conformal_CT_learner.predict_p_value(X_test, ite_test))
        list_p_values_y0.append(conformal_CT_learner.predict_p_value_y0(X_test, y0_test))
        list_p_values_y1.append(conformal_CT_learner.predict_p_value_y1(X_test, y1_test))
        list_p_values_oracle.append(conformal_ite_oracle.predict_cps(X_test, y=ite_test))

    dict_p_values = {
            "pseudo_MC_T": np.concatenate(list_p_values_pseudo_MC_T),
            "MC_T": np.concatenate(list_p_values_MC_T),
            # "pseudo_MC_S": np.concatenate(list_p_values_pseudo_MC_S),
            # "MC_S": np.concatenate(list_p_values_MC_S),
            # "pseudo_MC_X": np.concatenate(list_p_values_pseudo_MC_X),
            # "MC_X": np.concatenate(list_p_values_MC_X),
            "CT": np.concatenate(list_p_values_CT),
            "y0": np.concatenate(list_p_values_y0),
            "y1": np.concatenate(list_p_values_y1),
            "oracle": np.concatenate(list_p_values_oracle),"
    }
    df_p_values = pd.DataFrame(dict_p_values)
    if max_min_y:
        if heteroscedastic_epsilon:
            df_p_values.to_csv(f"../../results/outputs/nie_wager/setup{setup_name}/p_values/simulations_{setup_name}_{learner_name}_{normalized_conformal_name}_max_min_y_heteroscedastic_unweighted_p_values.csv", index=False)
        else:
            df_p_values.to_csv(f"../../results/outputs/nie_wager/setup{setup_name}/p_values/simulations_{setup_name}_{learner_name}_{normalized_conformal_name}_max_min_y_unweighted_p_values.csv", index=False)
    else:
        if heteroscedastic_epsilon:
            df_p_values.to_csv(f"../../results/outputs/nie_wager/setup{setup_name}/p_values/simulations_{setup_name}_{learner_name}_{normalized_conformal_name}_heteroscedastic_unweighted_p_values.csv", index=False)
        else:
            df_p_values.to_csv(f"../../results/outputs/nie_wager/setup{setup_name}/p_values/simulations_{setup_name}_{learner_name}_{normalized_conformal_name}_unweighted_p_values.csv", index=False)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]